In [1]:
import pymongo
from pymongo import MongoClient
import datetime as dt
# Method 1: connect on the default host and port
client = MongoClient () # method 1: connect on the default host and port
db= client.fit3182_assignment_db
collection = db.Mycollection

results = collection.find().sort('date', -1).limit(1)

for item in results:
    print(item)
    latestDate=item["date"]
latestDate = dt.datetime.strptime(latestDate,'%Y-%m-%d')
print(latestDate)
print(type(latestDate))


{'_id': ObjectId('5ee1a2ae29351fe33d723c3a'), 'latitude': -37.614000000000004, 'longitude': 143.451, 'air_temperature_celcius': 19, 'relative_humidity': 60.2, 'windspeed_knots': 6.4, 'max_wind_speed': 12.0, 'precipitation ': ' 0.00I', 'producer': 1, 'date': '2019-2-7', 'hotspot_histories': [{'latitude': -36.6952, 'longitude': 144.7228, 'confidence': 84.0, 'surface_temperature_celcius': 57.0, 'producer': 2, 'time': 'T11:19:1', 'datetime': '2019-2-7T11:19:1'}, {'latitude': -37.3046, 'longitude': 143.5448, 'confidence': 93.0, 'surface_temperature_celcius': 72.0, 'producer': 2, 'time': 'T11:19:3', 'datetime': '2019-2-7T11:19:3'}]}
2019-02-07 00:00:00
<class 'datetime.datetime'>


In [2]:
import datetime as dt
from time import sleep
from kafka import KafkaProducer
import pandas as pd
import json
import random


def publish_message(producer_instance, topic_name, key, value):
    try:
        if type(topic_name) == bytes:
            topic_name = topic_name.decode('utf-8')
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + value)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except IOError:
        print ("IO error")
    except ValueError:
        print ("Conversion error")
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer



if __name__ == "__main__":
    producer_id = 1
    key = str(producer_id)
    #topic = "streaming"
    topic = "climate"
    producer = connect_kafka_producer()
    climate_data  = pd.read_csv("/home/student/Documents/climate_streaming.csv") #change the path
    latestDateInDB =  latestDate #FIND THE LATEST DATE IN THE DB
    lastDatetime = dt.datetime.now()
    
    #make each record into json
    #for i in range(len(climate_data)):
    while True:
        randomint=random.randint(0,len(climate_data)-1)
        data = {}
        for j in climate_data:
            data[j] = climate_data[j][randomint]
        #extra information
        data["producer"] = producer_id
        #new day every 10 seconds
        currentDatetime = dt.datetime.now()
        duration = currentDatetime - lastDatetime
        if duration.total_seconds() >= 10:
            latestDateInDB = latestDateInDB + dt.timedelta(days=1)
            #THIS IS ASSUMING YOU COMPARE THE TEN SECONDS INTERVAL WITH THE LAST CREATED DATE
            #INSTEAD OF KEEP ADDING 10 SECONDS TO THE FIRST DATA'S CREATED DATE
            lastDatetime = currentDatetime
        data["date"] = str(latestDateInDB.year) + "-" + str(latestDateInDB.month) + "-" + str(latestDateInDB.day)
        
        #deserialize json and replace '' with ""
        deserialize_json = str(data).replace("\'", "\"")
        
        #send data
        publish_message(producer, topic, key, deserialize_json)
        sleep(10) #send data every 10 seconds
        #print(str(data))


Message published successfully. Data: {"latitude": -36.939999999999998, "longitude": 143.28100000000001, "air_temperature_celcius": 21, "relative_humidity": 52.799999999999997, "windspeed_knots": 4.0999999999999996, "max_wind_speed": 11.1, "precipitation ": " 0.00I", "producer": 1, "date": "2019-2-7"}
Message published successfully. Data: {"latitude": -37.472000000000001, "longitude": 148.119, "air_temperature_celcius": 15, "relative_humidity": 49.899999999999999, "windspeed_knots": 7.9000000000000004, "max_wind_speed": 13.0, "precipitation ": " 0.01G", "producer": 1, "date": "2019-2-8"}
Message published successfully. Data: {"latitude": -35.364000000000004, "longitude": 141.06299999999999, "air_temperature_celcius": 15, "relative_humidity": 51.899999999999999, "windspeed_knots": 6.5999999999999996, "max_wind_speed": 12.0, "precipitation ": " 0.01G", "producer": 1, "date": "2019-2-9"}
Message published successfully. Data: {"latitude": -36.134599999999999, "longitude": 145.2196000000000

Message published successfully. Data: {"latitude": -37.066899999999997, "longitude": 141.0556, "air_temperature_celcius": 12, "relative_humidity": 47.5, "windspeed_knots": 15.0, "max_wind_speed": 18.100000000000001, "precipitation ": " 0.20G", "producer": 1, "date": "2019-3-7"}
Message published successfully. Data: {"latitude": -37.326999999999998, "longitude": 148.083, "air_temperature_celcius": 10, "relative_humidity": 44.799999999999997, "windspeed_knots": 8.8000000000000007, "max_wind_speed": 14.0, "precipitation ": " 0.39G", "producer": 1, "date": "2019-3-8"}
Message published successfully. Data: {"latitude": -36.283999999999999, "longitude": 146.14500000000001, "air_temperature_celcius": 19, "relative_humidity": 54.100000000000001, "windspeed_knots": 11.199999999999999, "max_wind_speed": 18.100000000000001, "precipitation ": " 0.31G", "producer": 1, "date": "2019-3-9"}
Message published successfully. Data: {"latitude": -37.601999999999997, "longitude": 149.31100000000001, "air_te

Message published successfully. Data: {"latitude": -37.614000000000004, "longitude": 143.45099999999999, "air_temperature_celcius": 19, "relative_humidity": 60.200000000000003, "windspeed_knots": 6.4000000000000004, "max_wind_speed": 12.0, "precipitation ": " 0.00I", "producer": 1, "date": "2019-4-4"}
Message published successfully. Data: {"latitude": -35.325000000000003, "longitude": 143.49700000000001, "air_temperature_celcius": 13, "relative_humidity": 48.5, "windspeed_knots": 5.2999999999999998, "max_wind_speed": 11.1, "precipitation ": " 0.08G", "producer": 1, "date": "2019-4-5"}
Message published successfully. Data: {"latitude": -36.7179, "longitude": 142.25360000000001, "air_temperature_celcius": 11, "relative_humidity": 40.399999999999999, "windspeed_knots": 9.1999999999999993, "max_wind_speed": 15.0, "precipitation ": " 0.04G", "producer": 1, "date": "2019-4-6"}
Message published successfully. Data: {"latitude": -37.604999999999997, "longitude": 149.30200000000002, "air_temper

KeyboardInterrupt: 

In [12]:
import random
climate  = pd.read_csv("/home/student/Documents/climate_streaming.csv")

for i in range(len(climate)):
    randomint=random.randint(0,len(climate)-1)
    data={}
    for j in climate:
        data[j] = climate_data[j][randomint]
    print(data)
    break

#record=climate.sample()
#print(record)

#print("------------------------------------")
#record=climate.sample()
#print(record)
#print("-------------------------------------")
#record=climate.sample()
#print(record)

{'latitude': -37.380000000000003, 'longitude': 149.334, 'air_temperature_celcius': 16, 'relative_humidity': 48.200000000000003, 'windspeed_knots': 9.9000000000000004, 'max_wind_speed': 18.100000000000001, 'precipitation ': ' 0.01G'}
